In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "Simple App")

sc

<SparkContext master=local appName=Simple App>

In [20]:
sqlContext = SQLContext(sc)

In [9]:
df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true', inferschema='true') \
    .load('data/Uber-Jan-Feb-FOIL.csv')

df

DataFrame[dispatching_base_number: string, date: string, active_vehicles: int, trips: int]

## Ausgabe des automatisch generierten Schemas

In [8]:
df.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: integer (nullable = true)
 |-- trips: integer (nullable = true)



In [10]:
df.registerTempTable("uber")

In [18]:
distinct_bases = sqlContext.sql("select distinct dispatching_base_number from uber")
distinct_bases

DataFrame[dispatching_base_number: string]

In [19]:
distinct_bases.collect()

[Row(dispatching_base_number='B02512'),
 Row(dispatching_base_number='B02598'),
 Row(dispatching_base_number='B02682'),
 Row(dispatching_base_number='B02765'),
 Row(dispatching_base_number='B02617'),
 Row(dispatching_base_number='B02764')]

In [16]:
sqlContext.sql("""select distinct(`dispatching_base_number`), 
                                sum(`trips`) as cnt from uber group by `dispatching_base_number` 
                                order by cnt desc""").show()

+-----------------------+-------+
|dispatching_base_number|    cnt|
+-----------------------+-------+
|                 B02764|1914449|
|                 B02617| 725025|
|                 B02682| 662509|
|                 B02598| 540791|
|                 B02765| 193670|
|                 B02512|  93786|
+-----------------------+-------+



In [17]:
sqlContext.sql("""select distinct(`date`), 
                                sum(`trips`) as cnt from uber group by `date` 
                                order by cnt desc limit 5""").show()

+---------+------+
|     date|   cnt|
+---------+------+
|2/20/2015|100915|
|2/14/2015|100345|
|2/21/2015| 98380|
|2/13/2015| 98024|
|1/31/2015| 92257|
+---------+------+



In [ ]:
sc.stop()